## Example: Tiling gridded data to XYZ structure and OSM structure

This example will show how one can export a raster dataset to individual tiles in one of the stuctures mentioned in the title above. Normally this can be done via the [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html) and getting gridded data via DataCatalog.[get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html). For this example we'll use our own created dummy data.

In [1]:
# Imports
from hydromt import raster
from numpy import random
from os.path import join

# Create dummy data
shape = (1800, 3600)
da0 = raster.full_from_transform(
    transform=[0.004166666666666666, 0.0, 0.0, 0.0, -0.004166666666666667, 0.0],
    shape=shape,
    nodata=-9999,
    name="dummy_tile_data",
    crs=4326,
)

# Add some values to the raster
da0.values = random.randint(-5000, 5000, shape)

`da0` is gridded data as a xarray.DataArray object. 
With HydroMT a xarray.DataArray has some extra functionality via `.raster`
This extra functionality does include the ability to write a raster to a tile database (tiling).


### XYZ stucture
First let's have a look at the XYZ structure.

A xarray.DataArray is simple written to a tile database in XYZ structure via .raster.to_xyz



In [6]:
# Write the database in XYZ stucture
folder = 'tmpdir'
name = 'dummy_xyz'
da0.raster.to_xyz(
    root=join(folder, name),
    px=256,
    zoomlevels=[0, 1, 3, 4],
)

The database in the 'dummy_xyz' folder now contains 4 zoomlevels as defined above. 

zoomlevel 0 is at the scale of the xarray.DataArray (one on one). zoomlevel 1 is downscaled by a factor 2 compared to zoomlevel 0. zoomlevel 3 is downscaled by a factor of 8 compared to zoomlevel 0.

In the example above the amount of pixels per tile is determined at 1024 pixels by 1024 pixels. This resolution must NOT exceed the resolution of the total xarray.DataArray. In this example it should then be smaller than 21600.

A mosaic is created per zoomlevel of these tiles in a .vrt file.

At last, a .yml file is produced which can be read by the [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html) of HydroMT.

In [ ]:
# Imports
from hydromt import DataCatalog

# Load the yml into a DataCatalog
data_catalog = DataCatalog(join(folder, f'{name}.yml'))

# View the structure of the DataCatalog
data_catalog

With DataCatalog.[get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html) a raster (.vrt) can be retreived. In case of a tile database it can be done for a certain zoomlevel. E.g.

In [ ]:
# without zoom_level the highest res data is fetched
da = data_catalog.get_rasterdataset(name)
da

In [ ]:
# Request a raster from the Datacatolog
da = data_catalog.get_rasterdataset(name, zoom_level=1)

# View the data
da

### OSM structure
Second on our list is the OSM structure from [OpenStreetMap](https://www.openstreetmap.org).

A xarray.DataArray is simple written to a tile database in OSM structure via .raster.to_osm

In [ ]:
# Write the database in OSM stucture

name = 'dummy_osm'
da0.raster.to_osm(
    root=join(folder, name),
    zl=5,
)

We won't go in depth here concerning the structure according to osm.
For this, visit: https://wiki.openstreetmap.org/wiki/Zoom_levels and subsequent references.

However, there are three things to be considered:

    - The resolution of a tile cannot be defined. This is predetermined at 256 pixels by 256 pixels.
    - CRS is always set at EPSG:3857
    - Zoomlevel 0 in the OSM structure spans the entire globe

In [ ]:
# add the .yml to the DataCatalog
data_catalog.from_yml(join(folder, f'{name}.yml'))

# View the updated DataCatalog
data_catalog

Like the XYZ stucture, retrieve the gridded data via DataCatalog.[get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html).

In [ ]:
# Get a raster from the DataCatalog
da = data_catalog.get_rasterdataset(name, zoom_level=3)

# View the data
da